In [ ]:
!pip install openai dotenv
!pip install datasets

In [2]:
from datasets import load_dataset
dataset = load_dataset("maiurilorenzo/divina-commedia")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/459k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14233 [00:00<?, ? examples/s]

In [3]:
import pandas as pd
import re

def canto_to_number(canto):
    match = re.search(r'Canto (\w+)', canto)
    if match:
        roman = match.group(1)
        return roman_to_int(roman)
    return canto

def roman_to_int(roman):
    roman_numerals = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10,
        'XI': 11, 'XII': 12, 'XIII': 13, 'XIV': 14, 'XV': 15, 'XVI': 16, 'XVII': 17, 'XVIII': 18, 'XIX': 19, 'XX': 20,
        'XXI': 21, 'XXII': 22, 'XXIII': 23, 'XXIV': 24, 'XXV': 25, 'XXVI': 26, 'XXVII': 27, 'XXVIII': 28, 'XXIX': 29, 'XXX': 30,
        'XXXI': 31, 'XXXII': 32, 'XXXIII': 33, 'XXXIV': 34, 'XXXV': 35
    }
    return roman_numerals.get(roman, roman)


In [4]:
df = dataset["train"].to_pandas()
df['canto'] = df['canto'].apply(canto_to_number)

tercet_df = df.groupby(['volume', 'canto', 'tercet'])['text'].apply(lambda x: '\n'.join(x)).reset_index()

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [24]:
import openai
import json
import pandas as pd

# Set your OpenAI API key
openai.api_key = OPENAI_API_KEY  # Replace with your actual key

# Helper function to extract graph from tercet

def extract_graph_from_tercet(cantica, canto, versi, tercet, simulate=False, model="gpt-3.5-turbo"):
    prompt = f"""
Sei un esperto di analisi narrativa e di parsing semantico. Il tuo compito è convertire le terzine tratte dalla Divina Commedia di Dante in un grafo strutturato. Per ciascuna terzina, estrai le seguenti informazioni:

1. **Nodi:** Qualsiasi Persona, Luogo, Animale, Peccato o Verso esplicitamente o implicitamente menzionato.
2. **Relazioni:** Interazioni significative tra questi nodi, utilizzando i tipi di relazione ammessi.

## Tipi di nodo ammessi (e le loro proprietà):
- Verse (Verso): cantica, canto, numero_verso, testo, autore, tema
- Person (Persona): id, ruolo, identità_storica, versi, citazione, tema
- Location (Luogo): id, descrizione, versi, tema
- Animal (Animale): id, simbolismo, versi, descrizione, tema
- Sin (Peccato): id, cerchio, descrizione, punizione, versi, tema

## Tipi di relazione ammessi (e le loro proprietà):
- MENTIONS: source (Verso), target (qualsiasi nodo), numero_verso, canto
- APPEARS_IN: source (qualsiasi nodo), target (Verso), numero_verso, canto
- GUIDES: source (Persona), target (Persona), versi
- ENCOUNTERS: source (Persona), target (qualsiasi nodo), versi
- BLOCKS_PATH: source (Animale), target (Persona), versi
- DESCRIBES: source (Verso), target (qualsiasi nodo), versi
- SUFFERS: source (Persona), target (Peccato), versi
- PUNISHES: source (Peccato), target (Persona), versi
- LOCATED_IN: source (Luogo), target (Luogo)
- REPRESENTS: source (qualsiasi nodo), target (concetto astratto), descrizione
- SEEK: source (Persona), target (Persona), versi
- JUDGES: source (Persona), target (Persona), versi
- NEXT_VERSE: source (Verso), target (Verso)
- ASSOCIATED_WITH: source (Peccato), target (Luogo)
- CONTAINS: source (Canto), target (Verso)

---

## Dati in ingresso
Cantica: {cantica}
Canto: {canto}
Versi: {versi}
Terzina: {tercet}

---

## Formato di output
{{
    "tercet": "{tercet}",
    "nodes": [{{<ogni nodo estratto>}}],
    "relationships": [{{<ogni relazione estratta>}}]
}}

Esempio di nodo:
{{
    "id": "Virgilio",
    "type": "Person",
    "properties": {{
        "role": "Guida",
        "historical_identity": "Poeta romano",
        "verses": ["Inferno 1:61-90"],
        "theme": "Guida spirituale"
    }}
}}

Esempio di relazione:
{{
    "source": "Virgilio",
    "target": "Narratore",
    "type": "GUIDES",
    "properties": {{
        "verses": ["Inferno 1:61-90"]
    }}
}}

---

Ora elabora la seguente terzina:
Cantica: {cantica}
Canto: {canto}
Versi: {versi}
Terzina: {tercet}

Restituisci esclusivamente l'oggetto JSON, senza spiegazioni o commenti.
"""

    request_payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You must return output strictly as valid JSON. Do not include any text before or after the JSON object."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0
    }

    if simulate:
        response = simulated_openai_create(**request_payload)
    else:
        response = openai.chat.completions.create(**request_payload)

    extracted_graph = json.loads(response.choices[0].message.content)
    return extracted_graph


In [25]:
df_inferno = tercet_df[tercet_df['volume'] == 'Inferno']
df_inferno.tail()

,volume,canto,tercet,text
1591,Inferno,34,43,Luogo è là giù da Belzebù remoto\ntanto quanto...
1592,Inferno,34,44,d’un ruscelletto che quivi discende\nper la bu...
1593,Inferno,34,45,Lo duca e io per quel cammino ascoso\nintrammo...
1594,Inferno,34,46,"salimmo sù, el primo e io secondo,\ntanto ch’i..."
1595,Inferno,34,47,E quindi uscimmo a riveder le stelle.


In [26]:
canto_num = 1
canto_df = df_inferno[df_inferno['canto'] == canto_num]

# # Example DataFrame representing a full canto split into tercets
# data = {
#     "cantica": ["Inferno"] * 3,
#     "canto": [1] * 3,
#     "tercet_number": [1, 2, 3],
#     "tercet": [
#         "Nel mezzo del cammin di nostra vita\nmi ritrovai per una selva oscura,\nché la diritta via era smarrita.",
#         "Ahi quanto a dir qual era è cosa dura\nesta selva selvaggia e aspra e forte\nche nel pensier rinova la paura!",
#         "Tant’è amara che poco è più morte;\nma per trattar del ben ch’i’ vi trovai,\ndirò de l’altre cose ch’i’ v’ho scorte."
#     ]
# }

# canto_df = pd.DataFrame(data)

all_graphs = []
for _, row in canto_df.iterrows():
  print(f"Processing canto {canto_num}, tercet {row.tercet}")
  cantica = row.volume
  canto = row.canto
  tercet = row.text
  graph = extract_graph_from_tercet(cantica, canto, tercet)
  all_graphs.append(graph)

# Combine into a single structure (optional - you could also save as JSON per tercet)
with open(f"canto_{canto_num}_graph_40.json", "w", encoding="utf-8") as f:
    json.dump(all_graphs, f, indent=2, ensure_ascii=False)

print("Processed graph saved as canto_1_graph.json")


Processing canto 1, tercet 1
Processing canto 1, tercet 2
Processing canto 1, tercet 3
Processing canto 1, tercet 4
Processing canto 1, tercet 5
Processing canto 1, tercet 6
Processing canto 1, tercet 7
Processing canto 1, tercet 8
Processing canto 1, tercet 9
Processing canto 1, tercet 10
Processing canto 1, tercet 11
Processing canto 1, tercet 12
Processing canto 1, tercet 13
Processing canto 1, tercet 14
Processing canto 1, tercet 15
Processing canto 1, tercet 16
Processing canto 1, tercet 17
Processing canto 1, tercet 18
Processing canto 1, tercet 19
Processing canto 1, tercet 20
Processing canto 1, tercet 21
Processing canto 1, tercet 22
Processing canto 1, tercet 23
Processing canto 1, tercet 24
Processing canto 1, tercet 25
Processing canto 1, tercet 26
Processing canto 1, tercet 27
Processing canto 1, tercet 28
Processing canto 1, tercet 29
Processing canto 1, tercet 30
Processing canto 1, tercet 31
Processing canto 1, tercet 32
Processing canto 1, tercet 33
Processing canto 1,